In [71]:
import pandas as pd
import numpy as np

In [72]:
data = pd.read_excel("URINELOTS.xlsx")
#data = data.iloc[:,1:2]
data.head()

,S.No,Patients Name,Age,Gender,Date,UHID,RAMJA Results,AIIMS Result culture,E.coli,K. Pneumoniae,P.aurogenosa,A.baumnnii,P.mirabilis,E.faecalis,S. aureus,S. saprophyticus
0,1,Sahroj,33.0,F,13/02/2024,20220065432,Enterococcus faecalis,Enterococcus faecalis,NaN,NaN,-5.26086,-4.09794,-3.58480,-8.10731,-5.62819,NaN
1,2,Shyanu,70.0,F,15/02/2024,20240016182,NaN,Contamination,-6.63325,-10.1276,-8.30086,-13.22480,-11.94120,-13.64090,-14.93870,NaN
2,3,Saloni,19.0,F,15/02/2024,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...",E.coli,-15.30980,-10.0174,-9.09993,-8.30414,-5.06676,-5.42121,-8.77797,NaN
3,4,Ravita,34.0,F,15/02/2024,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...",CONTAMINATION,-7.70704,-8.0891,-4.79176,-8.63416,-11.84570,-9.31657,-9.90339,
4,5,Jyoti,28.0,F,15/02/2024,20201415102,NaN,NaN,0.00000,0.0000,0.00000,0.00000,0.00000,0.00000,0.00000,NaN


In [73]:
data.shape

(146, 16)

#### DATA cleaning

In [74]:
data = data.dropna(subset=['RAMJA Results','AIIMS Result culture'])

In [80]:
import re
def cleanup(text):
    text = text.upper()
    text = text.replace(".", " ")
    text = text.replace(" AND ", " ")
    text = re.sub(r'\s+', ' ', text)  # Use re.sub for regular expression replacement
    text = text.strip()
    return text


In [85]:
bacteriatypes = {
    "COLI" : 'Escherichia coli', 
    "PNEUMO" : 'Klebsiella pneumoniae',
    "AERUGI" : 'Pseudomonas aeruginosa',
    "BAUMAN" : 'Acinetobacter baumannii', 
    "MIRABIL" : 'Proteus mirabilis', 
    "FAECALI" : 'Enterococcus faecalis', 
    "AUREUS" : 'Staphylococcus aureus', 
    "SAPROPHY" : 'Staphylococcus saprophyticus',
    "STERI" : 'Sterile'
}

In [ ]:
""" import re

def cleanup(series):
    series = series.str.upper()
    series = series.str.replace(".", " ")
    series = series.str.replace(" AND ", " ")
    series = series.str.replace(r'\s+', ' ', regex=True)
    series = series.str.strip()
    return series """

#### RAMJA results

In [82]:
resultsRAMJA = data[['UHID','RAMJA Results']]

In [83]:
resultsRAMJA['cleanRAMJA'] = resultsRAMJA['RAMJA Results'].apply(cleanup)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_7664\766998183.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsRAMJA['cleanRAMJA'] = resultsRAMJA['RAMJA Results'].apply(cleanup)


In [84]:
resultsRAMJA.head()

,UHID,RAMJA Results,cleanRAMJA
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...","E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI..."
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...","K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA..."
5,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...","POLYMICROBIAL (E COLI, A BAUMANNII, P MIRABILI..."
6,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus","PSEUDOMONAS, P MIRABILIS, E FAECALIS, S AUREUS"


In [86]:
def check(mainstring, s1, s2):
    if (mainstring.count(s1) > 0 or mainstring.count(s2) > 0):
        #print("YES")
        return(1)
    else:
        #print("NO")
        return(0)


mainstring = "E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI"
s1 = "E COLI"
s2 = "K PNEUMONIAE"
check(mainstring, s2, s1)

1

In [87]:
genus = ["ESCHER","KLEBS","PSEUDO","ACINE","PROT","ENTER","AURE","SAPRO","STERI"]
species = ["COLI", "PNEUMO", "AERUGI", "BAUMAN", "MIRABIL", "FAECALI", "AUREUS", "SAPROPHY", "STERI"]
for s1, s2 in zip(genus,species):
    resultsRAMJA[s2] = resultsRAMJA['cleanRAMJA'].apply(lambda x: check(x, s1, s2))

C:\Users\neelesh\AppData\Local\Temp\ipykernel_7664\1057333925.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsRAMJA[s2] = resultsRAMJA['cleanRAMJA'].apply(lambda x: check(x, s1, s2))
C:\Users\neelesh\AppData\Local\Temp\ipykernel_7664\1057333925.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsRAMJA[s2] = resultsRAMJA['cleanRAMJA'].apply(lambda x: check(x, s1, s2))
C:\Users\neelesh\AppData\Local\Temp\ipykernel_7664\1057333925.py:4: SettingWithCopyWarning: 
A value is trying to be set on 

In [88]:
resultsRAMJA.head()

,UHID,RAMJA Results,cleanRAMJA,COLI,PNEUMO,AERUGI,BAUMAN,MIRABIL,FAECALI,AUREUS,SAPROPHY,STERI
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...","E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",1,1,1,1,0,0,1,0,0
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...","K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",0,1,0,1,1,1,1,0,0
5,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...","POLYMICROBIAL (E COLI, A BAUMANNII, P MIRABILI...",1,0,0,1,1,1,1,0,0
6,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus","PSEUDOMONAS, P MIRABILIS, E FAECALIS, S AUREUS",0,0,1,0,1,1,1,0,0


In [89]:
resultsRAMJA.rename(columns=bacteriatypes, inplace=True)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_7664\78547064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsRAMJA.rename(columns=bacteriatypes, inplace=True)


In [90]:
resultsRAMJA.head()

,UHID,RAMJA Results,cleanRAMJA,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0
2,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...","E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",1,1,1,1,0,0,1,0,0
3,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...","K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",0,1,0,1,1,1,1,0,0
5,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...","POLYMICROBIAL (E COLI, A BAUMANNII, P MIRABILI...",1,0,0,1,1,1,1,0,0
6,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus","PSEUDOMONAS, P MIRABILIS, E FAECALIS, S AUREUS",0,0,1,0,1,1,1,0,0


#### AIIMS results

In [91]:
resultsAIIMS = data[['UHID','AIIMS Result culture']]

In [92]:
resultsAIIMS['cleanAIIMS'] = resultsAIIMS['AIIMS Result culture'].apply(cleanup)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_7664\265187514.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsAIIMS['cleanAIIMS'] = resultsAIIMS['AIIMS Result culture'].apply(cleanup)


In [93]:
resultsAIIMS.head()

,UHID,AIIMS Result culture,cleanAIIMS
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS
2,20230125389,E.coli,E COLI
3,20180022685,CONTAMINATION,CONTAMINATION
5,20240012395,E.coli,E COLI
6,20220166372,E.coli & Pseudomnas,E COLI & PSEUDOMNAS


In [94]:
genus = ["ESCHER","KLEBS","PSEUDO","ACINE","PROT","ENTER","AURE","SAPRO","STERI"]
species = ["COLI", "PNEUMO", "AERUGI", "BAUMAN", "MIRABIL", "FAECALI", "AUREUS", "SAPROPHY", "STERI"]
for s1, s2 in zip(genus,species):
    resultsAIIMS[s2] = resultsAIIMS['cleanAIIMS'].apply(lambda x: check(x, s1, s2))
    #print(f"Element from array1: {s1}, Element from array2: {s2}")

C:\Users\neelesh\AppData\Local\Temp\ipykernel_7664\3468983276.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsAIIMS[s2] = resultsAIIMS['cleanAIIMS'].apply(lambda x: check(x, s1, s2))
C:\Users\neelesh\AppData\Local\Temp\ipykernel_7664\3468983276.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsAIIMS[s2] = resultsAIIMS['cleanAIIMS'].apply(lambda x: check(x, s1, s2))
C:\Users\neelesh\AppData\Local\Temp\ipykernel_7664\3468983276.py:4: SettingWithCopyWarning: 
A value is trying to be set on 

In [95]:
resultsAIIMS.head()

,UHID,AIIMS Result culture,cleanAIIMS,COLI,PNEUMO,AERUGI,BAUMAN,MIRABIL,FAECALI,AUREUS,SAPROPHY,STERI
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0
2,20230125389,E.coli,E COLI,1,0,0,0,0,0,0,0,0
3,20180022685,CONTAMINATION,CONTAMINATION,0,0,0,0,0,0,0,0,0
5,20240012395,E.coli,E COLI,1,0,0,0,0,0,0,0,0
6,20220166372,E.coli & Pseudomnas,E COLI & PSEUDOMNAS,1,0,1,0,0,0,0,0,0


In [96]:
resultsAIIMS.rename(columns=bacteriatypes, inplace=True)

C:\Users\neelesh\AppData\Local\Temp\ipykernel_7664\4113831117.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resultsAIIMS.rename(columns=bacteriatypes, inplace=True)


In [97]:
resultsAIIMS.head()

,UHID,AIIMS Result culture,cleanAIIMS,Escherichia coli,Klebsiella pneumoniae,Pseudomonas aeruginosa,Acinetobacter baumannii,Proteus mirabilis,Enterococcus faecalis,Staphylococcus aureus,Staphylococcus saprophyticus,Sterile
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0
2,20230125389,E.coli,E COLI,1,0,0,0,0,0,0,0,0
3,20180022685,CONTAMINATION,CONTAMINATION,0,0,0,0,0,0,0,0,0
5,20240012395,E.coli,E COLI,1,0,0,0,0,0,0,0,0
6,20220166372,E.coli & Pseudomnas,E COLI & PSEUDOMNAS,1,0,1,0,0,0,0,0,0


#### Combining both AIIMS and RAMJA results

In [103]:
results = pd.merge(resultsRAMJA, resultsAIIMS, on="UHID")

In [104]:
results.head()

,UHID,RAMJA Results,cleanRAMJA,Escherichia coli_x,Klebsiella pneumoniae_x,Pseudomonas aeruginosa_x,Acinetobacter baumannii_x,Proteus mirabilis_x,Enterococcus faecalis_x,Staphylococcus aureus_x,...,cleanAIIMS,Escherichia coli_y,Klebsiella pneumoniae_y,Pseudomonas aeruginosa_y,Acinetobacter baumannii_y,Proteus mirabilis_y,Enterococcus faecalis_y,Staphylococcus aureus_y,Staphylococcus saprophyticus_y,Sterile_y
0,20220065432,Enterococcus faecalis,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,...,ENTEROCOCCUS FAECALIS,0,0,0,0,0,1,0,0,0
1,20230125389,"E.Coli, K.pneumoniae, P. aeruginosa, A. bauman...","E COLI, K PNEUMONIAE, P AERUGINOSA, A BAUMANNI...",1,1,1,1,0,0,1,...,E COLI,1,0,0,0,0,0,0,0,0
2,20180022685,"K.pneumoniae, E. faecalis, P. mirabilis, A. b...","K PNEUMONIAE, E FAECALIS, P MIRABILIS, A BAUMA...",0,1,0,1,1,1,1,...,CONTAMINATION,0,0,0,0,0,0,0,0,0
3,20240012395,"Polymicrobial (E.coli, A. baumannii, P.mirabil...","POLYMICROBIAL (E COLI, A BAUMANNII, P MIRABILI...",1,0,0,1,1,1,1,...,E COLI,1,0,0,0,0,0,0,0,0
4,20220166372,"Pseudomonas, P.mirabilis, E.Faecalis, S. aureus","PSEUDOMONAS, P MIRABILIS, E FAECALIS, S AUREUS",0,0,1,0,1,1,1,...,E COLI & PSEUDOMNAS,1,0,1,0,0,0,0,0,0


In [105]:
#results.to_excel("URINELOT1.xlsx")

#### Compare few results

In [102]:
print("RAMJA : \n", resultsRAMJA["Escherichia coli"].value_counts())
print("AIIMS : \n",resultsAIIMS["Escherichia coli"].value_counts())

RAMJA : 
 0    75
1    43
Name: Escherichia coli, dtype: int64
AIIMS : 
 0    100
1     18
Name: Escherichia coli, dtype: int64
